In [1]:
import numpy as np 
import pandas as pd
import os, sys
from time import time
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers, callbacks
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,Convolution2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras import models, layers
import pandas as pd
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping


Using TensorFlow backend.


In [2]:
FileName = r"C:\Users\sidda\Desktop\plots\Figure13model\mapped_data\AllRunsmapped.csv"
df = pd.read_csv(FileName)
print(df.shape)
df.head()

(2569, 8)


,Address,Element,FileName,Recovery,Run,SheetName,Sl_No,Stage
0,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Lead,Cell1 19h 06m 48.7750s.jpg,7.548923,1,R1_Lead,2,1
1,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Lead,Cell1 19h 06m 58.5150s.jpg,8.758845,1,R1_Lead,3,1
2,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Lead,Cell1 19h 07m 10.1370s.jpg,9.605029,1,R1_Lead,4,1
3,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Lead,Cell1 19h 07m 20.7539s.jpg,10.123912,1,R1_Lead,5,1
4,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Lead,Cell1 19h 07m 32.409s.jpg,10.577631,1,R1_Lead,6,1


In [3]:
df = df[df.Element== 'Zinc']
print(df.shape)
df.head()

(1547, 8)


,Address,Element,FileName,Recovery,Run,SheetName,Sl_No,Stage
43,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 19h 31m 12.6240s.jpg,9.481437,1,R1_Zinc,57,2
44,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 19h 31m 22.7880s.jpg,11.687719,1,R1_Zinc,58,2
45,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 19h 31m 32.9070s.jpg,14.963232,1,R1_Zinc,59,2
46,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 19h 31m 46.1760s.jpg,16.307366,1,R1_Zinc,60,2
47,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 19h 31m 57.2850s.jpg,17.622741,1,R1_Zinc,61,2


In [4]:
#Setting and Cropping image files
   #Set Image crop dimensions- Set it to 128 * 128
#Set image attributes
left=115
top=35
right=243
bottom=163

In [5]:
#Splitting data into training, validation, testing based on runs and then cropping each image

#Training Data- Runs 2,5,7,9

df_train = df.copy()
df_train = df_train[df_train.Run==2]
df_train=df_train.append(df[df.Run==5])
df_train=df_train.append(df[df.Run==7])
df_train=df_train.append(df[df.Run==9])



train_files = []
y_train = []
train_dataset = []
for index, row in df_train.iterrows():
    y_train.append(row['Recovery']) 
    img_train = load_img(row['Address'])  # this is a PIL image
    
    img_train_cropped = img_train.crop((left,top,right,bottom))
    
    #img2 =img1.thumbnail((128, 128))
    x_train = img_to_array(img_train_cropped)
    #x = x.reshape((3, 128, 128))
    #x_train = (x_train - 128.0) / 128.0
    x_train = (x_train - 255) / 255
    train_dataset.append(x_train)
    
print("training dataset taken")
print(df_train.shape)
print(len(y_train))

df_train.head()

training dataset taken
(739, 8)
739


,Address,Element,FileName,Recovery,Run,SheetName,Sl_No,Stage
120,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 14h 55m 34.7750s.jpg,16.891826,2,R2_Zinc,171,2
121,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 14h 55m 44.8180s.jpg,20.469347,2,R2_Zinc,172,2
122,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 14h 55m 55.1219s.jpg,21.911156,2,R2_Zinc,173,2
123,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 14h 56m 05.1380s.jpg,22.205057,2,R2_Zinc,174,2
124,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 14h 57m 56.8180s.jpg,27.804938,2,R2_Zinc,185,2


In [7]:
#Validation Data-1,4,8

df_val = df.copy()
df_val = df_val[df_val.Run==4]
df_val=df_val.append(df[df.Run==8])
#df_val=df_val.append(df[df.Run==8])



val_files = []
y_val = []
val_dataset = []
for index, row in df_val.iterrows():
    y_val.append(row['Recovery']) 
    img_val = load_img(row['Address'])  # this is a PIL image
    
    img_val_cropped = img_val.crop((left,top,right,bottom))
    
    #img2 =img1.thumbnail((128, 128))
    x_val = img_to_array(img_val_cropped)
    #x = x.reshape((3, 128, 128))
    #x_val = (x_val - 128.0) / 128.0
    x_val = (x_val - 255) / 255
    val_dataset.append(x_val)
    
print("Validation dataset taken")
print(df_val.shape)
print(len(y_val))

df_val.head()

Validation dataset taken
(455, 8)
455


,Address,Element,FileName,Recovery,Run,SheetName,Sl_No,Stage
488,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 20h 00m 22.3269s.jpg,8.039598,4,R4_Zinc,592,2
489,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 20h 00m 24.4019s.jpg,8.454997,4,R4_Zinc,593,2
490,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 20h 00m 26.4330s.jpg,8.998775,4,R4_Zinc,594,2
491,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 20h 00m 28.4879s.jpg,9.725356,4,R4_Zinc,595,2
492,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 20h 00m 30.5559s.jpg,10.174955,4,R4_Zinc,596,2


In [8]:
#Testing- Runs 3,6

df_test = df.copy()
df_test = df_test[df_test.Run==1]
df_test=df_test.append(df[df.Run==3])
df_test=df_test.append(df[df.Run==6])


test_files = []
y_test = []
test_dataset = []
for index, row in df_test.iterrows():
    y_test.append(row['Recovery']) 
    img_test = load_img(row['Address'])  # this is a PIL image
    
    img_test_cropped = img_test.crop((left,top,right,bottom))
    
    #img2 =img1.thumbnail((128, 128))
    x_test = img_to_array(img_test_cropped)
    #x = x.reshape((3, 128, 128))
    #x_test = (x_test - 128.0) / 128.0
    x_test = (x_test - 255) / 255
    test_dataset.append(x_test)
    
print("Testing dataset taken")
print(df_test.shape)
print(len(y_test))

df_test.head()

Testing dataset taken
(353, 8)
353


,Address,Element,FileName,Recovery,Run,SheetName,Sl_No,Stage
43,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 19h 31m 12.6240s.jpg,9.481437,1,R1_Zinc,57,2
44,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 19h 31m 22.7880s.jpg,11.687719,1,R1_Zinc,58,2
45,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 19h 31m 32.9070s.jpg,14.963232,1,R1_Zinc,59,2
46,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 19h 31m 46.1760s.jpg,16.307366,1,R1_Zinc,60,2
47,C:\Users\sidda\Desktop\plots\Figure13model\Cle...,Zinc,Cell1 19h 31m 57.2850s.jpg,17.622741,1,R1_Zinc,61,2


In [13]:
# CNN Architecture
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(128,128,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'linear'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 63, 63, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 30, 64)        256       
________________________________________________

In [15]:
model.compile(loss=rmse, optimizer='rmsprop', metrics=['mae'])
#callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [16]:
#len(X_train)

In [18]:
#model.fit([train_dataset], y_train, batch_size = 32, epochs = 5)

model.fit([train_dataset], y_train, batch_size=32,epochs=200,validation_data=([val_dataset], y_val))


Train on 739 samples, validate on 455 samples
Epoch 1/200
739/739 [==============================] - 19s 26ms/step - loss: 15.4555 - mae: 13.2423 - val_loss: 25.7684 - val_mae: 25.6135
Epoch 2/200
739/739 [==============================] - 18s 25ms/step - loss: 10.6411 - mae: 8.5396 - val_loss: 21.1378 - val_mae: 20.9275
Epoch 3/200
739/739 [==============================] - 19s 25ms/step - loss: 10.2520 - mae: 8.2732 - val_loss: 18.1671 - val_mae: 17.8926
Epoch 4/200
739/739 [==============================] - 18s 24ms/step - loss: 10.2808 - mae: 8.3248 - val_loss: 12.6173 - val_mae: 12.4238
Epoch 5/200
739/739 [==============================] - 21s 29ms/step - loss: 9.8516 - mae: 8.0065 - val_loss: 11.9653 - val_mae: 11.7839
Epoch 6/200
739/739 [==============================] - 23s 32ms/step - loss: 9.9021 - mae: 7.9319 - val_loss: 5.3253 - val_mae: 4.7257
Epoch 7/200
739/739 [==============================] - 29s 39ms/step - loss: 9.7503 - mae: 7.8412 - val_loss: 6.2516 - val_mae: 5

Epoch 61/200
739/739 [==============================] - 22s 29ms/step - loss: 7.2914 - mae: 5.8785 - val_loss: 12.2378 - val_mae: 12.0283
Epoch 62/200
739/739 [==============================] - 21s 29ms/step - loss: 7.2251 - mae: 5.8014 - val_loss: 4.4635 - val_mae: 3.8786
Epoch 63/200
739/739 [==============================] - 22s 29ms/step - loss: 7.5782 - mae: 6.0453 - val_loss: 9.7346 - val_mae: 9.5026
Epoch 64/200
739/739 [==============================] - 21s 29ms/step - loss: 7.3194 - mae: 5.8261 - val_loss: 10.0146 - val_mae: 9.7130
Epoch 65/200
739/739 [==============================] - 21s 29ms/step - loss: 7.4007 - mae: 5.8696 - val_loss: 2.9673 - val_mae: 2.2202
Epoch 66/200
739/739 [==============================] - 21s 29ms/step - loss: 7.1401 - mae: 5.7913 - val_loss: 8.5165 - val_mae: 8.2030
Epoch 67/200
739/739 [==============================] - 23s 32ms/step - loss: 7.5847 - mae: 5.9443 - val_loss: 2.9195 - val_mae: 2.4281
Epoch 68/200
739/739 [=======================

Epoch 121/200
739/739 [==============================] - 17s 23ms/step - loss: 7.0175 - mae: 5.5616 - val_loss: 5.1090 - val_mae: 4.5009
Epoch 122/200
739/739 [==============================] - 16s 22ms/step - loss: 6.9838 - mae: 5.5917 - val_loss: 4.8098 - val_mae: 4.2334
Epoch 123/200
739/739 [==============================] - 16s 22ms/step - loss: 6.6523 - mae: 5.3423 - val_loss: 6.9138 - val_mae: 6.5315
Epoch 124/200
739/739 [==============================] - 16s 21ms/step - loss: 7.0911 - mae: 5.6328 - val_loss: 6.2799 - val_mae: 5.8523
Epoch 125/200
739/739 [==============================] - 16s 21ms/step - loss: 6.9775 - mae: 5.5398 - val_loss: 7.9650 - val_mae: 7.6341
Epoch 126/200
739/739 [==============================] - 16s 22ms/step - loss: 7.0238 - mae: 5.5983 - val_loss: 2.7413 - val_mae: 2.1916
Epoch 127/200
739/739 [==============================] - 17s 23ms/step - loss: 7.0943 - mae: 5.7393 - val_loss: 5.4782 - val_mae: 5.0092
Epoch 128/200
739/739 [==================

Epoch 181/200
739/739 [==============================] - 20s 27ms/step - loss: 6.4968 - mae: 5.2296 - val_loss: 7.6707 - val_mae: 7.1742
Epoch 182/200
739/739 [==============================] - 19s 26ms/step - loss: 6.6013 - mae: 5.1931 - val_loss: 5.4031 - val_mae: 4.7749
Epoch 183/200
739/739 [==============================] - 18s 24ms/step - loss: 6.5596 - mae: 5.2260 - val_loss: 2.9195 - val_mae: 2.2277
Epoch 184/200
739/739 [==============================] - 20s 27ms/step - loss: 6.3818 - mae: 5.0146 - val_loss: 4.7233 - val_mae: 3.7796
Epoch 185/200
739/739 [==============================] - 17s 23ms/step - loss: 6.5788 - mae: 5.1429 - val_loss: 10.2251 - val_mae: 9.8213
Epoch 186/200
739/739 [==============================] - 19s 25ms/step - loss: 6.7649 - mae: 5.4171 - val_loss: 9.2625 - val_mae: 8.8832
Epoch 187/200
739/739 [==============================] - 18s 24ms/step - loss: 6.5487 - mae: 5.1890 - val_loss: 3.5503 - val_mae: 2.8554
Epoch 188/200
739/739 [=================

In [19]:
df.head()
print(df.shape)

(1547, 8)


In [20]:
model.evaluate([test_dataset], y_test,batch_size=16)

353/353 [==============================] - 4s 10ms/step


[3.303848169342992, 2.9013302326202393]

In [21]:
predictions_test = model.predict([test_dataset])

In [22]:
#Finding the R-squared values of Regression
from sklearn.metrics import r2_score
Rsquared_testset=r2_score(y_test, predictions_test)
print(Rsquared_testset)

0.4596199306678649


In [23]:
#Training and Validation Rsquared
predictions_train= model.predict([train_dataset])
predictions_val=model.predict([val_dataset])
Rsquared_trainset=r2_score(y_train, predictions_train)
Rsquared_valset=r2_score(y_val, predictions_val)
print(Rsquared_trainset)
print(Rsquared_valset)


0.4787252100343037
0.15983217786023107
